In [9]:
# processing us_project_data.csv to make it ready
# for analysis by addressing data quality issues

In [10]:
import pandas as pd

In [11]:
df = pd.read_csv("~/Desktop/us_project_data.csv")

In [12]:
# Team 1
#
# Remove the "Total" rows found in the "Unique Investment Identifier"
# column
index_rows = df[df["Unique Investment Identifier"] == "Total" ].index
df.drop(index_rows, inplace = True)


In [13]:
# Team 2
#
# There are several columns with dates including:
# "Start Date"
# "Completion Date (B1)"
# "Planned Project Completion Date (B2)"
# "Projected/Actual Project Completion Date (B2)"
# "Updated Date"
#
# Each of these dates appears to be formatted day/month/year
# The standard way to represent dates within pandas is
# "yyyy-mm-dd"
# 
# Add a new column for each of these variables and use
# the new variable name mappings below:
#
# There are several columns with dates including:
# "Start Date" => "start_date"
# "Completion Date (B1)" => "completion_date"
# "Planned Project Completion Date (B2)" => "planned_date"
# "Projected/Actual Project Completion Date (B2)" => "projected_date"
# "Updated Date" => "updated_date"
#
# parse start_date to DateTime object
df['start_date'] = pd.to_datetime(df['Start Date']) 
dates = []

# manually handle this column bc of inconsistent formatting
for d in df['Completion Date (B1)'].values: 
    
    # Try except blocks run code in the try block. If the code errors, it will run the except block.
    try:
        dates.append(pd.to_datetime(d,dayfirst=True))
    except:
        dates.append(pd.to_datetime(d,yearfirst=True))
    
df['completion_date'] = dates

# Continue the same procedure for the rest of the columns
df['planned_date'] = pd.to_datetime(df['Planned Project Completion Date (B2)'])
df['projected_date'] = pd.to_datetime(df['Projected/Actual Project Completion Date (B2)'])
df['updated_date'] = pd.to_datetime(df['Updated Date'])



In [14]:
df.head()

,Unique Investment Identifier,Business Case ID,Agency Code,Agency Name,Investment Title,Project ID,Agency Project ID,Project Name,Project Description,Start Date,...,Planned Cost ($ M),Projected/Actual Cost ($ M),Updated Date,Updated Time,Unique Project ID,start_date,completion_date,planned_date,projected_date,updated_date
0,005-000001723,212.0,5.0,Department of Agriculture,AMS Infrastructure WAN and DMZ (AMSWAN),656.0,NaN,Operations,Annual Agency Operations.,01/10/2011,...,15.2970,15.2970,22/09/2011,10:22:25,1,2011-01-10,2012-09-30,NaT,NaT,2011-09-22
1,005-000001723,212.0,5.0,Department of Agriculture,AMS Infrastructure WAN and DMZ (AMSWAN),657.0,NaN,Virtualization,Program Areas will migrate their data over to ...,01/10/2011,...,0.1790,0.1790,30/11/2011,06:09:57,2,2011-01-10,2012-03-31,2012-03-31,2012-03-31,2011-11-30
2,005-000001723,212.0,5.0,Department of Agriculture,AMS Infrastructure WAN and DMZ (AMSWAN),658.0,NaN,Refresh,Programs Areas will replace 1/3 of their compu...,01/04/2012,...,1.4600,1.4600,28/10/2011,05:50:19,3,2012-01-04,2012-09-30,NaT,NaT,2011-10-28
3,005-000001822,213.0,5.0,Department of Agriculture,APHIS Electronic Permits System (ePermits),661.0,NaN,ePermits O&M FY11 Part 1.,"Production Support including Analysis, Softwar...",01/04/2011,...,1.8205,1.4564,31/05/2012,14:20:11,PER1822001,2011-01-04,2011-09-30,2011-09-30,2011-09-30,2012-05-31
4,005-000001822,213.0,5.0,Department of Agriculture,APHIS Electronic Permits System (ePermits),662.0,NaN,ePermits O&M FY12 Part 1,"Production Support including Analysis, Softwar...",01/04/2012,...,1.7130,1.7130,31/05/2012,14:20:11,PER1822002,2012-01-04,2012-09-30,2012-09-30,2012-09-30,2012-05-31


In [18]:
# Team 3
#
# Reformat the string columns to remove content that may be
# problematic for natural language processing later.  Go through
# the following colums:
#
# "Investment Title"
# "Project Name"
# "Project Description"
# 
# remove instances of "$", "/", "\", "&amp", "&"
# be sure to keep case, ".", "(", ")", ";", and ","
# create new variables according to the mapping below:
#
# "Investment Title" => "title"
# "Project Name" => "name"
# "Project Description" => "description"

df['title'] = df['Investment Title'].str.replace('$', '').str.replace('/', '').str.replace('\\', '').str.replace('&', '').str.replace('&amp', '')

df['name'] = df['Project Name'].str.replace('$', '').str.replace('/', '').str.replace('\\', '').str.replace('&', '').str.replace('&amp', '')

df['description'] = df['Project Description'].str.replace('$', '').str.replace('/', '').str.replace('\\', '').str.replace('&', '').str.replace('&amp', '')


In [20]:
# Team 4
#
# Create a completed column based on the updated date
#
# The new column in the dataframed called "completed"
# is a binary variable that is 1 if the "updated_date"
# is later than "projected_date" based on Team 2's processing.
# The idea is to have a variable to know if the project is
# ongoing (i.e. completed=0) or done (i.e. completed=1)

df['start_date'] = pd.to_datetime(df['start_date'])
df['updated_date'] = pd.to_datetime(df['updated_date'])
for index,row in df.iterrows():
    if df.at[index,'start_date'] < df.at[index,'updated_date']:
        df.at[index,'completed'] = 1
    else:
        df.at[index,'completed'] = 0


In [21]:
# Team 5
#
# Create dummy variables for each agency below.  The value
# in that column is "1" if the agency is true and "0" if it
# is not that agency
# Department of Agriculture
# Department of Commerce
# Department of Defense
# Department of Education
# Department of Energy
# Department of Health and Human Services
# Department of Homeland Security
# Department of Housing and Urban Development
# Department of Justice
# Department of Labor
# Department of State
# Department of the Interior
# Department of the Treasury
# Department of Transportation
# Department of Veterans Affairs
# Environmental Protection Agency
# General Services Administration
# National Aeronautics and Space Administration
# National Archives and Records Administration
# National Science Foundation
# Nuclear Regulatory Commission
# Office of Personnel Management
# Small Business Administration
# Social Security Administration
# U.S. Agency for International Development
# U.S. Army Corps of Engineers
#

departments = ['Department of Agriculture', 'Department of Commerce', 'Department of Defense' 'Department of Education', 'Department of Energy', 'Department of Health and Human Services', 'Department of Homeland Security', 'Department of Housing and Urban Development', 'Department of Justice', 'Department of Labor', 'Department of State', 'Department of the Interior', 'Department of the Treasury', 'Department of Transportation', 'Department of Veterans Affairs', 'Environmental Protection Agency', 'General Services Administration', 'National Aeronautics and Space Administration', 'National Archives and Records Administration', 'National Science Foundation', 'Nuclear Regulatory Commission', 'Office of Personnel Management', 'Small Business Administration', 'Social Security Administration', 'U.S. Agency for International Development', 'U.S. Army Corps of Engineers']

for dept in departments:
    df[dept] = [1 if agency == dept else 0 for agency in df['Agency Name']]

    

In [22]:
df.head()

,Unique Investment Identifier,Business Case ID,Agency Code,Agency Name,Investment Title,Project ID,Agency Project ID,Project Name,Project Description,Start Date,...,General Services Administration,National Aeronautics and Space Administration,National Archives and Records Administration,National Science Foundation,Nuclear Regulatory Commission,Office of Personnel Management,Small Business Administration,Social Security Administration,U.S. Agency for International Development,U.S. Army Corps of Engineers
0,005-000001723,212.0,5.0,Department of Agriculture,AMS Infrastructure WAN and DMZ (AMSWAN),656.0,NaN,Operations,Annual Agency Operations.,01/10/2011,...,0,0,0,0,0,0,0,0,0,0
1,005-000001723,212.0,5.0,Department of Agriculture,AMS Infrastructure WAN and DMZ (AMSWAN),657.0,NaN,Virtualization,Program Areas will migrate their data over to ...,01/10/2011,...,0,0,0,0,0,0,0,0,0,0
2,005-000001723,212.0,5.0,Department of Agriculture,AMS Infrastructure WAN and DMZ (AMSWAN),658.0,NaN,Refresh,Programs Areas will replace 1/3 of their compu...,01/04/2012,...,0,0,0,0,0,0,0,0,0,0
3,005-000001822,213.0,5.0,Department of Agriculture,APHIS Electronic Permits System (ePermits),661.0,NaN,ePermits O&M FY11 Part 1.,"Production Support including Analysis, Softwar...",01/04/2011,...,0,0,0,0,0,0,0,0,0,0
4,005-000001822,213.0,5.0,Department of Agriculture,APHIS Electronic Permits System (ePermits),662.0,NaN,ePermits O&M FY12 Part 1,"Production Support including Analysis, Softwar...",01/04/2012,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# Team 6
#
# Create a new variable called "flag"
# the default value of this variable is 0 but it
# will change to 1 if a potential issue or error is
# found in processing the data

df['flag'] = 0


In [27]:
# Team 7
#
# Check to see if the number of days between "planned_date"
# and "projected" date is the same as the column
# "Schedule Variance (in days)".  There are a few conditions
# that might be the case:
#
# if "planned_date" and/or "projected_date" is missing,
# do not do the computation and assume whatever is reported
# in "Schedule Variance (in days)" is correct; leave the
# "flag" variable alone
# 
# if "planned_date" and "projected_date" is there, check
# to see if the difference between these two is the same
# as the value reported in "Schedule Variance (in days)".
# if it is the same, leave the "flag" variable alone.
# However, if the difference is not the same as what is
# reported, change the "flag" variable to 1
#
# next, the team should check to see that the value reported
# in "Schedule Variance (%)" is correct as well
# this should be the "Schedule Variance (in days)" divided
# by "completion_date" - "start_date".  If it is not, then
# change the "flag" variable to 1

def planned_projected(row):
  flag = row['flag']
  if row['Planned Project Completion Date (B2)'] != '' and row['Projected/Actual Project Completion Date (B2)'] != '':
    diff = (pd.to_datetime(row['Planned Project Completion Date (B2)']) - pd.to_datetime(row['Projected/Actual Project Completion Date (B2)'])).days
    if diff != row['Schedule Variance (in days)']:
      flag = 1 
  
  return flag
 
flags = df.apply(lambda x: planned_projected(x), axis=1)
df['flag'] = flags


In [28]:
# Team 8
#
# Confirm that the computations are correct for the following
# variables:
# 
# "Cost Variance ($ M)" = 
# "Projected/Actual Cost ($ M)" - Planned Cost ($ M)"
#
# "Cost Variance (%)" - 
# "Cost Variance ($ M")/"Planned Cost ($ M)"
#
# if these things are not true, then change the flag bit to 1
# for this observation
for index, row in df.iterrows():
    
    if abs(row["Cost Variance ($ M)"] - (row["Projected/Actual Cost ($ M)"] - row["Planned Cost ($ M)"])) > 0.1:
        df.at[index, 'flag'] = 1
    elif abs(row["Cost Variance (%)"] - (row["Cost Variance ($ M)"] / row["Planned Cost ($ M)"])) > 0.1:
        df.at[index, 'flag'] = 1


In [29]:
# Team 9
#
# Make sure that the values contained in "Unique Project ID"
# Are, in fact, unique.  If they are not unique, then change
# the flag bit to 1
#
unique_ids = []
counts = df["Unique Project ID"].value_counts()
        
for i, rr in df.iterrows():
    if (counts[rr['Unique Project ID']] >1):
        
        #df[i]['Unique Project ID']= 1
        df.at[i, "flag"]=1



In [30]:
# Now that we have processed all of the data, how many
# rows in the dataframe have a flag of 0 and how many
# have a flag of 1?
print("Number of non-flagged rows: ",len(df[df['flag']==0]))
print("Number of flagged rows: ",len(df[df['flag']==1]))

Number of non-flagged rows:  396
Number of flagged rows:  2072
